In [1]:
import re
import requests
import csv
from bs4 import BeautifulSoup as bs

In [2]:
template ='https://news.search.yahoo.com/search?p={}'
url = template.format('iphone12+leaked')

In [3]:
headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

In [4]:
response = requests.get(url,headers=headers)

In [5]:
soup = bs(response.text,'html.parser')

In [6]:
cards = soup.find_all('div', 'NewsArticle')

In [7]:
len(cards)

10

create a prototype model

In [8]:
card = cards[0]

In [9]:
headlines = card.find('h4','s-title').text

In [10]:
source = card.find('span','s-source').text

In [11]:
posted = card.find('span','s-time').text.replace('·','').strip()

In [12]:
describtion = card.find('p','s-desc').text.strip()

In [13]:
raw_url = card.find('a').get('href')

In [14]:
raw_url

'https://r.search.yahoo.com/_ylt=AwrC1CgWmFxgRScACjHQtDMD;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1616709782/RO=10/RU=https%3a%2f%2fwww.tomsguide.com%2fnews%2fiphone-13-leak-just-revealed-the-smaller-notch-weve-been-waiting-for/RK=2/RS=2nAdYMkcxqFiCkw8MtsTLbQd4RM-'

In [15]:
unquoted_link = requests.utils.unquote(raw_url)

In [16]:
unquoted_link

'https://r.search.yahoo.com/_ylt=AwrC1CgWmFxgRScACjHQtDMD;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1616709782/RO=10/RU=https://www.tomsguide.com/news/iphone-13-leak-just-revealed-the-smaller-notch-weve-been-waiting-for/RK=2/RS=2nAdYMkcxqFiCkw8MtsTLbQd4RM-'

In [17]:
raw_url

'https://r.search.yahoo.com/_ylt=AwrC1CgWmFxgRScACjHQtDMD;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1616709782/RO=10/RU=https%3a%2f%2fwww.tomsguide.com%2fnews%2fiphone-13-leak-just-revealed-the-smaller-notch-weve-been-waiting-for/RK=2/RS=2nAdYMkcxqFiCkw8MtsTLbQd4RM-'

In [18]:
pattern = re.compile(r'RU=(.+)\/RK')

In [19]:
clean_link = re.search(pattern,unquoted_link).group(1)

In [20]:
clean_link

'https://www.tomsguide.com/news/iphone-13-leak-just-revealed-the-smaller-notch-weve-been-waiting-for'

get next page

In [21]:
url = soup.find('a','next').get('href')

In [22]:
url

'https://news.search.yahoo.com/search;_ylt=AwrC1CgWmFxgRScAHjHQtDMD;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3BhZ2luYXRpb24-?p=iphone12+leaked&b=11&pz=10&bct=0&xargs=0'

generalize the model

In [103]:
def get_article(card):
    headlines = card.find('h4','s-title').text
    source = card.find('span','s-source').text
    posted = card.find('span','s-time').text.replace('·','').strip()
    describtion = card.find('p','s-desc').text.strip()
    raw_url = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_url)
    clean_link = re.search(pattern,unquoted_link).group(1)
    
    article = (headlines,source,posted,describtion,clean_link)
    return article 

In [104]:
articles = []
links = set()
for card in cards:
    article = get_article(card)
    link = article[-1]
    if not link in links:
        links.add(link)
        articles.append(article)

bring it all together

In [105]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    
    article = (headline, source, posted, description, clean_link)
    return article

def get_the_news(search):
    """Run the main program"""
    template = 'https://news.search.yahoo.com/search?p={}'
    url = template.format(search)
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')
        
        # extract articles from page
        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        # find the next page
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            break
            
    # save article data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Source', 'Posted', 'Description', 'Link'])
        writer.writerows(articles)
        
    return articles

In [107]:
iphone_12 = get_the_news('iphone 12 leaked')

In [109]:
iphone_12[:5]

[("iPhone 13 rumors so far: What we're hearing about release date, specs, price and more",
  'CNET',
  '3 days ago',
  "Apple's iPhone 12 lineup has only been on virtual shelves for a few months, and already the mobile...",
  'https://www.cnet.com/news/apple-iphone-13-rumors-so-far-what-were-hearing-about-release-date-specs-price-and-more/'),
 ('OnePlus 9 — 5 biggest camera upgrades revealed',
  "Tom's Guide",
  '5 days ago',
  'Leading up to the OnePlus 9 launch, information about its new revamped camera system continues to leak online. The OnePlus 8T showed that the company...',
  'https://www.tomsguide.com/news/oneplus-9-5-biggest-camera-upgrades-revealed'),
 ("Apple Promotes iPhone 12 'Ceramic Shield' Display In New 'Cook' Ad",
  'MacRumors',
  '4 days ago',
  'A new set of renders claims to showcase the design of the upcoming third-generation AirPods, which...',
  'https://www.macrumors.com/2021/03/13/iphone-12-cook-ad/'),
 ('Why Apple Is Accusing Former Employee Of Leaking Produc